In [ ]:

# Make Predictions with k-nearest neighbors on the Iris Flowers Dataset
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

'''# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 '''
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

# Make a prediction with KNN on Iris Dataset
'''filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)'''
# define model parameter
num_neighbors = 5
# define a new record
row = [5.7,2.9,4.2,1.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

In [ ]:
!nvidia-smi -q -i 0 | grep "Product Name"

    Product Name                          : Tesla P100-PCIE-16GB


In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 127 µs (started: 2021-12-10 15:36:28 +00:00)


In [ ]:
!pip install nufhe[pycuda]
!pip install pycuda

     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 191 kB 11.2 MB/s 
     |████████████████████████████████| 1.7 MB 61.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 66 kB 5.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627693 sha256=994d3d80b596ee9b1f97a67b809d3f9d55529c36bb77c564cbd9f56863bd1a44
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=1b3a85806cb4962a183ee7a69342ab217fc821e5e5ff26818e862401e27cde00
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
  Created wheel for reikna: filename=reikna-0.7.6-py3-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 30.1 s (started: 2021-12-10 15:38:30 +00:00)


# kNN breast cancer




In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime

def cba4(r,a,b,carry):
    temp= vm.gate_and(a, b) #ab
    temp1= vm.gate_and( b,carry ) #bc
    temp2= vm.gate_and(a, carry) # ac
    temp3= vm.gate_or(temp, temp1) #ab+ac
    r[1]=vm.gate_or(temp3,temp2) # ab+abc+ac
    r[0]=vm.gate_not(r[1]) #(ab+bc+ca)'
    return r
def cba1(r,a,b,carry):
    temp= vm.gate_and(b,carry) #temp=bc
    r[1]=vm.gate_or(temp, a)   #r1=bc+a
    r[0]=vm.gate_not(r[1])     #r0=(bc+a)'
    return r

def threetotwo(r,p0,p1,p2):
    r[0] = vm.gate_or(p0, p1)
    temp=vm.gate_and(p0, p1)
    r[1]=vm.gate_or(temp, p2)
    return r
def twotoone(r, p0,p1 ):
    r[0] = vm.gate_or(p0, p1)
    return r

def subtractBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)      #axorb
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry) #axorbxorc
    # And(t1[0], t1[0], t2[0])
    acomp=vm.gate_not(a)   #a'
    abcomp=vm.gate_not(t1)  #axorb'
    t2 = vm.gate_and(acomp, b)
    t3 = vm.gate_and(abcomp, carry)
    #t4=vm.gate_and(t2,t3)
    #t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t2, t3)

    return r
def subtractNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(0,nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    t1=vm.gate_not(ctA[0])
    carry = vm.gate_and(t1, ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        bitResult = subtractBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])

        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes

def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
#def bubblesort(distances)

def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row,nbits)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors




def euclidean_distance(row1, row2,nBits):
    ones= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    distance=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    #distance = 0.0
    '''for i in range(nBits):
        temp=vm.gate_copy(row2[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)'''


    result=subtractNumbers(row1,row2,nBits)
    signbit=vm.gate_copy(result[len(result)-1])
    for i in range(nBits):
        temp=vm.gate_copy(result[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)
    for i in range(0,nBits):
           distance[i]=vm.gate_mux(signbit,neg2[i],result[i])
    return distance



def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''


    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
        pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r
def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])

    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}"
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s):
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')

    #print(listp)
    str1 = ""
    # traverse in the string
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s:
        str1 += ele
    # return string
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)


    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

def compare_bit(  a, b,  lsb_carry,  tmp):
    result= ctx.encrypt(secret_key, [False])
    tmp=vm.gate_xnor(a, b)
    result=vm.gate_mux(tmp,lsb_carry, a)
    return result


def minimum(  a,  b,  nb_bits):
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
        tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1



def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=



    for i in range(output_bits):
        ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])
    # Copy(carry[0], bitResult[1])
    return ctRes


def dist_calc(line,line1,nbits):
    #print(line)
    print(type(list))

    return wx,b_l














if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(size)]
    temp_dist=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_label=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=vm.gate_copy(one)
    with open('/content/drive/MyDrive/pc11train.txt') as f:
        lines = []
        for line in f:
            lines.append([int(v) for v in line.split()])
    with open('/content/drive/MyDrive/pc11test.txt') as f:
        lines1 = []
        for line in f:
            lines1.append([int(v) for v in line.split()])
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    train_size= int(input("Please enter train a string:\n"))
    print("training size",train_size)
    test_size=int(input("Please enter test data a string:\n"))
    print("test size",test_size)
    neigh=int(input("Please enter no of neighbours:\n"))
    print("neighbours",neigh)

    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    #w_p=[-15,-18 ,-78 ,138 , 62] # scratch x/1+|x|
    w_p=[-1,-3,4,2,0] # sk learn x/1+|x|
    #w_p=[4,-8 ,22 ,9 , -1] sigmoid
    #w_p=[-3, -4 , 10,  6, -1] svm

    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    '''w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)'''
    '''eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
        bias[i]=ctx.encrypt(secret_key, bias_b[i])
        eten[i]=ctx.encrypt(secret_key, eten_b[i])
    '''
    #train_data = pd.read_csv('/home/pradeep/Desktop/train.txt',sep=" ",header = None)
    #test_data=pd.read_csv('/home/pradeep/Desktop/test.txt',sep=" ",header = None)
    dist=[]
    for i in range(train_size):
        dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)])

    plain_predict=[]
    start_time = datetime.now()
    print("start time",start_time)
    #dist=[[vm.empty_ciphertext((1,))] for i in range(train_size) for j in range(size)]
    for j in range(test_size):
        print("j values is",j)
        print("################################")
        ciphertext1a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext2a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext3a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext4a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext5a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext6a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext7a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext8a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext9a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext10a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext11a=[[vm.empty_ciphertext((1,))] for i in range(size)]
        temp1=int(lines1[j][0])
        b_y0=twos_complement(temp1,size)
        temp1=int(lines1[j][1])
        b_y1=twos_complement(temp1,size)
        temp1=int(lines1[j][2])
        b_y2=twos_complement(temp1,size)
        temp1=int(lines1[j][3])
        b_y3=twos_complement(temp1,size)
        temp1=int(lines1[j][4])
        b_y4=twos_complement(temp1,size)
        temp1=int(lines1[j][5])
        b_y5=twos_complement(temp1,size)
        temp1=int(lines1[j][6])
        b_y6=twos_complement(temp1,size)
        temp1=int(lines1[j][7])
        b_y7=twos_complement(temp1,size)
        temp1=int(lines1[j][8])
        b_y8=twos_complement(temp1,size)
        temp1=int(lines1[j][9])
        b_y9=twos_complement(temp1,size)
        temp1=int(lines1[j][10])
        b_y10=twos_complement(temp1,size)

        for k in range(size):
          ciphertext1a[k] = ctx.encrypt(secret_key, b_y0[k])
          ciphertext2a[k] = ctx.encrypt(secret_key, b_y1[k])
          ciphertext3a[k] = ctx.encrypt(secret_key, b_y2[k])
          ciphertext4a[k] = ctx.encrypt(secret_key, b_y3[k])
          ciphertext5a[k] = ctx.encrypt(secret_key, b_y4[k])
          ciphertext6a[k] = ctx.encrypt(secret_key, b_y5[k])
          ciphertext7a[k] = ctx.encrypt(secret_key, b_y6[k])
          ciphertext8a[k] = ctx.encrypt(secret_key, b_y7[k])
          ciphertext9a[k] = ctx.encrypt(secret_key, b_y8[k])
          ciphertext10a[k] = ctx.encrypt(secret_key, b_y9[k])
          ciphertext11a[k] = ctx.encrypt(secret_key, b_y10[k])


        for i in range(train_size):
            print("i value is ",i)
            print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4])
            print(lines1[j][0],lines1[j][1],lines1[j][2],lines1[j][3])
            temp=int(lines[i][0])
            b_x0=twos_complement(temp,size)
            temp=int(lines[i][1])
            b_x1=twos_complement(temp,size)
            temp=int(lines[i][2])
            b_x2=twos_complement(temp,size)
            temp=int(lines[i][3])
            b_x3=twos_complement(temp,size)
            temp=int(lines[i][4])
            b_x4=twos_complement(temp,size)
            temp=int(lines[i][5])
            b_x5=twos_complement(temp,size)
            temp=int(lines[i][6])
            b_x6=twos_complement(temp,size)
            temp=int(lines[i][7])
            b_x7=twos_complement(temp,size)
            temp=int(lines[i][8])
            b_x8=twos_complement(temp,size)
            temp=int(lines[i][9])
            b_x9=twos_complement(temp,size)
            temp=int(lines[i][10])
            b_x10=twos_complement(temp,size)


            ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext5=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext6=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext7=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext8=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext9=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext10=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext11=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertextl=[[vm.empty_ciphertext((1,))] for i in range(size)]

            for _ in range(size):
              ciphertext1[_] = ctx.encrypt(secret_key, b_x0[_])
              ciphertext2[_] = ctx.encrypt(secret_key, b_x1[_])
              ciphertext3[_] = ctx.encrypt(secret_key, b_x2[_])
              ciphertext4[_] = ctx.encrypt(secret_key, b_x3[_])
              ciphertext5[_] = ctx.encrypt(secret_key, b_x4[_])
              ciphertext6[_] = ctx.encrypt(secret_key, b_x5[_])
              ciphertext7[_] = ctx.encrypt(secret_key, b_x6[_])
              ciphertext8[_] = ctx.encrypt(secret_key, b_x7[_])
              ciphertext9[_] = ctx.encrypt(secret_key, b_x8[_])
              ciphertext10[_] = ctx.encrypt(secret_key, b_x9[_])
              ciphertext11[_] = ctx.encrypt(secret_key, b_x10[_])
            #ciphertext1 = ctx.encrypt(secret_key, x)
            #ciphertext2 = ctx.encrypt(secret_key, y)
            start_time = time.time()
            #ciphertext1.reverse()
            #ciphertext2.reverse()
            #result = addNumbers(ciphertext1, ciphertext2, size)
            #partial_predict=[]

            #print(type(ciphertext1[0][0]))
            temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            #wx=[[vm.empty_ciphertext((1,))] for i in range(size)]
            #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
            #temp1=[]
            #temp2=[]
            presult_mul1 = euclidean_distance(ciphertext1,ciphertext1a, size)
            #print("ciphertext one",type(ciphertext1))
            #print("ciphertext one inside",type(ciphertext1[0]))
            '''temp1=presult_mul1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul1",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul2 = euclidean_distance(ciphertext2,ciphertext2a , size)
            '''temp1=presult_mul2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul2",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul3 = euclidean_distance(ciphertext3,ciphertext3a,  size )
            '''temp1=presult_mul3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul3",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul4 = euclidean_distance(ciphertext4, ciphertext4a,  size)
            presult_mul5 = euclidean_distance(ciphertext5, ciphertext5a,  size)
            presult_mul6 = euclidean_distance(ciphertext6, ciphertext6a,  size)
            presult_mul7 = euclidean_distance(ciphertext7, ciphertext7a,  size)
            presult_mul8 = euclidean_distance(ciphertext8, ciphertext8a,  size)
            presult_mul9 = euclidean_distance(ciphertext9, ciphertext9a,  size)
            presult_mul10 = euclidean_distance(ciphertext10, ciphertext10a,  size)
            presult_mul11 = euclidean_distance(ciphertext11, ciphertext11a,  size)

            '''temp1=presult_mul4[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul4",twos_comp_val(int(pa,2),len(pa)))'''
            presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
            '''temp1=presult_add1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("firstadd",pa)
            print("add1",twos_comp_val(int(pa,2),len(pa)))'''
            presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
            presult_add3 = addNumbers(presult_mul5, presult_mul6,  size)
            presult_add4 = addNumbers(presult_mul7, presult_mul8,  size)
            presult_add5 = addNumbers(presult_mul9, presult_mul10,  size)
            #second reduction
            presult_add21 = addNumbers(presult_add1, presult_add2,  size)
            presult_add22 = addNumbers(presult_add3, presult_add4,  size)
            presult_add23 = addNumbers(presult_add5, presult_mul11,  size)
            #third reduction
            presult_add31 = addNumbers(presult_add21, presult_add22,  size)
            '''temp1=presult_add2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("secondadd",pa)
            print("add2",twos_comp_val(int(pa,2),len(pa)))'''


            dist[i][0]= addNumbers(presult_add31, presult_add23,  size)
            temp1=dist[i][0][:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("secondadd",pa)
            print("distance is",twos_comp_val(int(pa,2),len(pa)))
            #print(type(temp))
            #x = twos_complement(deci_x,size)
            for k in range(size):
              dist[i][1][k]=vm.gate_copy(ciphertextl[k])
        for l in range( train_size):
            # Last i elements are already in place
            for m in range(train_size-l-1):
                #if (arr[j] > arr[j+1]):
                #swap(&arr[j], &arr[j+1]);
                temp=subtractNumbers(dist[m][0],dist[m+1][0], size)
                '''temp1=temp[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("temp",twos_comp_val(int(pa,2),len(pa)))'''

                signbit= ctx.encrypt(secret_key, [False])
                signbitnot= ctx.encrypt(secret_key, [False])
                signbit=temp[size-1]

                signbitnot=vm.gate_not(signbit)
                tempb1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb2=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb3=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb4=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempn1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempn2=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempn3=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempn4=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                for _ in range(size):
                    tempb1[_]=dist[m][0][_]
                    tempb2[_]=dist[m+1][0][_]
                    tempb3[_]=dist[m][1][_]
                    tempb4[_]=dist[m+1][1][_]
                '''temp1=tempb1[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("tempb1",twos_comp_val(int(pa,2),len(pa)))
                temp1=tempb2[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("tempb2",twos_comp_val(int(pa,2),len(pa)))
                temp1=tempb3[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                #print("tempb3",pa)
                print(" tempb3",twos_comp_val(int(pa,2),len(pa)))
                temp1=tempb4[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                #print("wx is",pa)
                print(" tempb4",twos_comp_val(int(pa,2),len(pa)))'''

                for k in range(size):
                    dist[m][0][k]=vm.gate_mux(signbit,dist[m][0][k],dist[m+1][0][k])
                    dist[m+1][0][k] =vm.gate_mux(signbit,tempb2[k],tempb1[k])
                    dist[m][1][k]=vm.gate_mux(signbit,dist[m][1][k],dist[m+1][1][k])
                    dist[m+1][1][k]=vm.gate_mux(signbit,tempb3[k],tempb4[k])

        pos=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        neg=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        #predict1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        tempp=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        posc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        negc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        signbitl= ctx.encrypt(secret_key, [False])
        signbitc= ctx.encrypt(secret_key, [False])
        for i in range(neigh):
            signbitl=vm.gate_copy(dist[i][1][size-1])
            signplain=[]
            signplain.append([ctx.decrypt(secret_key,signbitl)])
            print("signbit of label ",signplain)
            signbitc=vm.gate_copy(onep[size-1])
            signplain=[]
            signplain.append([ctx.decrypt(secret_key,signbitc)])
            print("signbit of 1",signplain)
            dbit=vm.gate_xnor(signbitl,signbitc)

            posc=pos[:]
            negc=neg[:]
            pos=make_neg(pos,size)
            neg=make_neg(neg,size)
            for j in range(size):
                pos[j]=vm.gate_mux(dbit,pos[j],posc[j])
                neg[j]=vm.gate_mux(dbit,negc[j],neg[j])
        temp1=pos[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("poscount",twos_comp_val(int(pa,2),len(pa)))
        temp1=neg[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("neg count",twos_comp_val(int(pa,2),len(pa)))
        temp1=temp[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("pos-neg",twos_comp_val(int(pa,2),len(pa)))

        temp=subtractNumbers(pos,neg,size)
        signbit= vm.gate_copy(temp[size-1])
        for i in range(size):
            tempp[i]=vm.gate_mux(signbit,onen[i],onep[i])
        #predict1=tempp[:]
        #predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        tempp.reverse()
        result_bits = [ctx.decrypt(secret_key, tempp[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        print(pa)
        print("preddicted",twos_comp_val(int(pa,2),len(pa)))
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print("j values is",j,"endtime",datetime.now(),"start time",start_time)
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])

Please enter train a string:
30
training size 30
Please enter test data a string:
20
test size 20
Please enter no of neighbours:
7
neighbours 7
start time 2021-12-10 15:44:31.586270
j values is 0
################################
i value is  0
-41 -2 10 -1 -6
-17 13 0 8


Exception ignored in: <function Thread.__del__ at 0x7fb5d66c3830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/cuda.py", line 247, in __del__
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/api.py", line 597, in release
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/cuda.py", line 242, in _release_specific
pycuda._driver.LogicError: context::pop failed: invalid device context - cannot pop non-current context


distance is 115


AttributeError: ignored

time: 1min 50s (started: 2021-12-10 15:44:12 +00:00)


# SVM breast cancer

In [ ]:
import time
import random
import nufhe
import numpy as np
from datetime import timedelta
def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out
def approaddBits(r, a, b, carry):

    '''# Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])po
    r[1] = vm.gate_or(t5, t4)'''
    '''r[1]= vm.gate_or(a, b)    #p0 + p1
    temp=vm.gate_and(a, b)
    r[0]=vm.gate_or(carry, temp)   #p0 p1 + p2
    #r[1]=vm.gate_or(w0, w1)
    #r[1] = vm.gate_and(w0, w1)'''
    '''#3to2
    r[0] = vm.gate_or(a, b)
    temp=vm.gate_and(a, b)
    r[1]=vm.gate_or(temp, carry)'''
    #2to1
    r[0] = vm.gate_or(a, b)
    '''#cba1
    temp= vm.gate_and(b,carry)
    r[1]=vm.gate_or(temp, a)
    r[0]=vm.gate_not(r[1])'''
    '''#cba2
    temp= vm.gate_and(a, b)
    r[1]=vm.gate_xor(temp, carry)
    r[0]=vm.gate_not(r[1])'''
    '''#cba3
    temp= vm.gate_and(a, b)
    r[1]=vm.gate_or(temp, carry)
    r[0]=vm.gate_not(r[1])'''
    '''#cba4
    temp= vm.gate_and(a, b)
    temp1= vm.gate_and( b,carry)
    temp2= vm.gate_and(a, carry)
    temp3= vm.gate_or(temp, temp1)
    r[1]=vm.gate_or(temp3,temp2)
    r[0]=vm.gate_not(r[1])'''
    '''# Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)'''

    return r

def addBits(r, a, b, carry):
    '''# Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])po
    r[1] = vm.gate_or(t5, t4)'''
    '''r[1]= vm.gate_or(a, b)    #p0 + p1
    temp=vm.gate_and(a, b)
    r[0]=vm.gate_or(carry, temp)   #p0 p1 + p2
    #r[1]=vm.gate_or(w0, w1)
    #r[1] = vm.gate_and(w0, w1)'''


    '''#cba1
    temp= vm.gate_and(b,carry)
    r[1]=vm.gate_or(temp, a)
    r[0]=vm.gate_not(r[1])'''
    '''#cba2
    temp= vm.gate_and(a, b)
    r[1]=vm.gate_xor(temp, carry)
    r[0]=vm.gate_not(r[1])'''
    '''#cba3
    temp= vm.gate_and(a, b)
    r[1]=vm.gate_or(temp, carry)
    r[0]=vm.gate_not(r[1])
    #cba4
    temp= vm.gate_and(a, b)
    temp1= vm.gate_and( b,carry)
    temp2= vm.gate_and(a, carry)
    temp3= vm.gate_or(temp, temp1)
    r[1]=vm.gate_or(temp3,temp2)
    r[0]=vm.gate_not(r[1])'''
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r


def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i<4:
          bitResult = approaddBits(bitResult, ctA[i], ctB[i], carry)
          # Copy(ctRes[i], bitResult[0]);
          ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
          #carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
          bitResult =addBits(bitResult, ctA[i], ctB[i], carry)
          # Copy(ctRes[i], bitResult[0]);
          ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
          carry = nufhe.LweSampleArray.copy(bitResult[1])
    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]

    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])
            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])

    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}"
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s):
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')

    #print(listp)
    str1 = ""
    # traverse in the string
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s:
        str1 += ele
    # return string
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)


    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp):
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits):
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    for i in range(output_bits):
      ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])
    # Copy(carry[0], bitResult[1])
    return ctRes

if __name__ == '__main__':
   with open('/content/drive/MyDrive/pc11.txt') as f:
    lines = []
    for line in f:
        lines.append([int(v) for v in line.split()])
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    test_size = int(input("Please enter a string:\n"))
    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    # 0.18989317 -0.25418705  0.02612883  0.00184397 -0.12274228 -0.01697259 -0.01228785 -0.06611546 -0.08922483  0.16153068 -0.02537609
    #w_p=[-3, -4 , 10,  6, -1]
    w_p=[-4, 6, -1, -1, 3, 1, 1, 2, 2, -4, 1, 6]
    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=12
    w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[len(w_p)-1],size)
    for i in range(size):
      bias[i]=ctx.encrypt(secret_key, bias_b[i])

    plain_predict=[]
    start_time = time.time()
    for i in range(test_size):
        print(lines[i][0],lines[i][1],lines[i][2],lines[i][3])
        temp=int(lines[i][0])
        b_x0=twos_complement(temp,size)
        temp=int(lines[i][1])
        b_x1=twos_complement(temp,size)
        temp=int(lines[i][2])
        b_x2=twos_complement(temp,size)
        temp=int(lines[i][3])
        b_x3=twos_complement(temp,size)
        temp=int(lines[i][4])
        b_x4=twos_complement(temp,size)
        temp=int(lines[i][5])
        b_x5=twos_complement(temp,size)
        temp=int(lines[i][6])
        b_x6=twos_complement(temp,size)
        temp=int(lines[i][7])
        b_x7=twos_complement(temp,size)
        temp=int(lines[i][8])
        b_x8=twos_complement(temp,size)
        temp=int(lines[i][9])
        b_x9=twos_complement(temp,size)
        temp=int(lines[i][10])
        b_x10=twos_complement(temp,size)



        ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext5=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext6=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext7=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext8=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext9=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext10=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext11=[[vm.empty_ciphertext((1,))] for i in range(size)]


        for j in range(size):
            ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
            ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
            ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
            ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
            ciphertext5[j] = ctx.encrypt(secret_key, b_x4[j])
            ciphertext6[j] = ctx.encrypt(secret_key, b_x5[j])
            ciphertext7[j] = ctx.encrypt(secret_key, b_x6[j])
            ciphertext8[j] = ctx.encrypt(secret_key, b_x7[j])
            ciphertext9[j] = ctx.encrypt(secret_key, b_x8[j])
            ciphertext10[j] = ctx.encrypt(secret_key, b_x9[j])
            ciphertext11[j] = ctx.encrypt(secret_key, b_x10[j])

        #ciphertext1 = ctx.encrypt(secret_key, x)
        #ciphertext2 = ctx.encrypt(secret_key, y)
        start_time = time.time()
        #ciphertext1.reverse()
        #ciphertext2.reverse()
        #result = addNumbers(ciphertext1, ciphertext2, size)
        #partial_predict=[]

        #print(type(ciphertext1[0][0]))
        temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
        #temp1=[]
        #temp2=[]
        presult_mul1 = mulNumbers(ciphertext1,w[0], secret_key, size, size * 2)
        '''temp1=presult_mul1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul2 = mulNumbers(ciphertext2, w[1], secret_key, size, size * 2)
        '''temp1=presult_mul2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul2",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul3 = mulNumbers(ciphertext3, w[2], secret_key, size, size * 2)
        '''temp1=presult_mul3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul3",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul4 = mulNumbers(ciphertext4, w[3], secret_key, size, size * 2)
        '''temp1=presult_mul4[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul4",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul5 = mulNumbers(ciphertext5, w[4], secret_key, size, size * 2)
        presult_mul6 = mulNumbers(ciphertext6, w[5], secret_key, size, size * 2)
        presult_mul7 = mulNumbers(ciphertext7, w[6], secret_key, size, size * 2)
        presult_mul8 = mulNumbers(ciphertext8, w[7], secret_key, size, size * 2)
        presult_mul9 = mulNumbers(ciphertext9, w[8], secret_key, size, size * 2)
        presult_mul10 = mulNumbers(ciphertext10, w[9], secret_key, size, size * 2)
        presult_mul11 = mulNumbers(ciphertext11, w[10], secret_key, size, size * 2)


        presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
        '''temp1=presult_add1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("firstadd",pa)
        print("add1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
        '''temp1=presult_add2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("secondadd",pa)
        print("add2",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add3 = addNumbers(presult_mul5, presult_mul6,  size)
        presult_add4 = addNumbers(presult_mul7, presult_mul8,  size)
        presult_add5 = addNumbers(presult_mul9, presult_mul10,  size)
        #first step
        presult_add11 = addNumbers(presult_add1, presult_add2,  size)
        presult_add12 = addNumbers(presult_add3, presult_add4,  size)
        presult_add13 = addNumbers(presult_add5, presult_mul11,  size)
        #secong step
        presult_add21 = addNumbers(presult_add11, presult_add12,  size)
        presult_add22 = addNumbers(presult_add13, bias,  size)
        '''temp1=presult_add3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("thirdadd",pa)
        print("add3",twos_comp_val(int(pa,2),len(pa)))'''


        partial_predict = addNumbers(presult_add21,presult_add22,  size)
        #partial_predict.revesrse()
        result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("partial predict",twos_comp_val(int(pa,2),len(pa)))
        predict1=predict(partial_predict,secret_key,size)
        predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))

        pa=listToString(result_bits)
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print(" nuFHE multiplication number is : ", plain_predict[i])
        if i%5==0:
          print("i value is ",i,str(timedelta(time.time() - start_time)))

    print("multiplication time",time.time() - start_time)
    #result.reverse()
    pos=0
    neg=0
    ''' for i in range(test_size):
      if int(lines[i][4])==int(plain_predict[j]):
        pos=pos+1
      else:
        neg=neg+1
    print("acc is =",pos/test_size)'''
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][11],"predicted", plain_predict[j])

Please enter a string:
1
81 4 -25 -26
partial predict -418
 nuFHE multiplication number is :  -1
i value is  0 914 days, 16:35:25.057983
multiplication time 914.6913578510284
 nuFHE multiplication number is actual -1 predicted -1
time: 15min 24s (started: 2021-12-10 09:35:43 +00:00)


In [ ]:
import os
from tensorflow.python.profiler import profiler_client

tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
print(profiler_client.monitor(tpu_profile_service_address, 100, 2))

  Timestamp: 13:45:46
  TPU type: TPU v2
  Utilization of TPU Matrix Units (higher is better): 0.000%


time: 1.97 s (started: 2021-11-06 13:45:44 +00:00)


In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()
vm = ctx.make_virtual_machine(cloud_key)
size=16
dist=[]
for i in range(2):
        dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)])
print(dist[0][1])

[<nufhe.lwe.LweSampleArray object at 0x7fac845cee20>, <nufhe.lwe.LweSampleArray object at 0x7fac6803f7f0>, <nufhe.lwe.LweSampleArray object at 0x7fac48589dc0>, <nufhe.lwe.LweSampleArray object at 0x7fac48310eb0>, <nufhe.lwe.LweSampleArray object at 0x7fac483fb9a0>, <nufhe.lwe.LweSampleArray object at 0x7fac48537bb0>, <nufhe.lwe.LweSampleArray object at 0x7fac6801fd00>, <nufhe.lwe.LweSampleArray object at 0x7fac48555100>, <nufhe.lwe.LweSampleArray object at 0x7fac484bb280>, <nufhe.lwe.LweSampleArray object at 0x7fac68014130>, <nufhe.lwe.LweSampleArray object at 0x7fac680145b0>, <nufhe.lwe.LweSampleArray object at 0x7fac845fcfd0>, <nufhe.lwe.LweSampleArray object at 0x7fac48417d60>, <nufhe.lwe.LweSampleArray object at 0x7fac48417670>, <nufhe.lwe.LweSampleArray object at 0x7fac48417eb0>, <nufhe.lwe.LweSampleArray object at 0x7fac485ce580>]
